# Analisis de Imagenes para texto alternativo usando contexto

In [1]:
import json
import boto3
from IPython.display import display, HTML
import base64

### 1. Crea el cliente Bedrock usando SDK de AWS

In [2]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'

bedrock = boto3.client("bedrock-runtime")


### 3. Analiza la imagen y crea el texto alternativo

In [10]:
image_path = "./samples/sample_alarm.jpg"

with open(image_path, "rb") as image_file:
    content_image = base64.b64encode(image_file.read()).decode('utf8')


previous_text = "En la imagen de abajo se ve las diferentes alarmas del sistema"
following_text = "Podemos ver que existen dos alarmas posibles."


message = {
    "role": "user",
    "content": [
        {"type":"text","text":previous_text},
        {
            "type": "image", 
            "source": {
                "type": "base64", 
                "media_type": "image/jpeg", 
                "data": content_image
            }
        },
        {"type":"text","text":following_text}
    ]
}

system_prompt = """Tu eres un revisor de articulos web que van a ser publicados, tu misión es ver las imágenes y leer el texto para encontrar contenido web no inclusivo para personas con discapacidad visual. A continuación te muestro algunos ejemplos:

Ejemplo de escritura no inclusiva:
Incorrecto:'Como se puede ver en la imagen de arriba, el proceso inicia...' 
Corrección: 'En el diagrama anterior, el proceso inicia...'

Ejemplo de imagen no inclusiva:
Incorrecto: (usando una imagen con colores rojo y verde) 'El color rojo representa un sistema alarmado, y el verde el sistema saludable'
Correcto: (usando una imagen con colores rojo y amarillo y etiquetas de texto para cada color) 'Acá se muestra el sistema alarmado y el sistema saludable'
"""


body = {
    "system": system_prompt,
    "messages":[message],"anthropic_version":"bedrock-2023-05-31","max_tokens":500, "temperature":0
}

response = bedrock.invoke_model(body=json.dumps(body), modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

alt_text = response_body.get("content")[0].get("text")

### 4. Resultado

In [11]:
print(response_body.get("content")[0].get("text"))

La descripción "En la imagen de abajo se ve las diferentes alarmas del sistema" no es inclusiva para personas con discapacidad visual, ya que asume que el lector puede ver la imagen. Una descripción más inclusiva sería:

La imagen muestra dos círculos de colores, uno rojo y otro verde, que representan diferentes tipos de alarmas del sistema. Sin embargo, no se proporcionan etiquetas de texto que indiquen qué significa cada color, lo cual dificultaría la comprensión para personas con discapacidad visual. Una mejor práctica sería incluir una leyenda o descripción textual que explique el significado de los colores utilizados.
